In [4]:
import os
import tempfile
import re
from math import floor
from simtk.openmm import app
from simtk import unit, openmm
from perses.utils.rbd import edit_tleap_in_inputs

import logging 

# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


In [6]:
tleap_prefix = "/data/chodera/zhangi/perses_benchmark/repex/31/10/0/debug/1_rbd_tleap"
tleap_in_temp = "temp"
tleap_out_temp = "temp.out"
edit_tleap_in_inputs(f"{tleap_prefix}.in", tleap_in_temp)
os.system(f"tleap -s -f {tleap_in_temp}.in > {tleap_out_temp}")



7936

Note: realized that this issue was because I was not properly copying the mutant PDB to the current dir

In [ ]:
# Retrieve charge and num of waters
with open(tleap_out_temp, "r") as f:
    lines_out = f.readlines()

for line in lines_out:
    if "Total unperturbed charge" in line:
        charge = float(line.split(":")[1].strip('\n'))
    if "residues" in line:
        result = re.findall(r"\d*", line)
        result_filtered = [r for r in result if r]
        num_waters = int(result_filtered[0])



In [ ]:
 # Compute number of ions (copied from OpenMM)
    numWaters = num_waters
    numPositive = 0
    numNegative = 0 
    totalCharge = charge
    ionicStrength = 0.15

    if totalCharge > 0:
        numNegative += totalCharge
    else:
        numPositive -= totalCharge

    numIons = (numWaters - numPositive - numNegative) * ionicStrength / (55.4)  # Pure water is about 55.4 molar (depending on temperature)
    numPairs = int(floor(numIons + 0.5))
    numPositive += numPairs
    numNegative += numPairs
    print(f"num positive: {numPositive}")
    print(f"num negative: {numNegative}")

    # Edit tleap file
    with open(f"{tleap_prefix}.in", "r") as f:
        lines_in = f.readlines()

    new_lines = []
    for line in lines_in:
        if "addionsrand complex" in line:
            line = f"addionsrand complex Na+ {int(numPositive)} Cl- {int(numNegative)}\n"
        new_lines.append(line)

    with open(f"{tleap_prefix}.in", 'w') as f:
        f.writelines(new_lines)
        